In [1]:
using DataFrames, TableOperations, Tables, Random
using Recommender: Movielens100k, load_dataset, ratio_split, ItemkNN, evaluate_u2i, MeanPrecision, MeanRecall, MeanNDCG

In [2]:
ml100k = Movielens100k()
download(ml100k)
rating, user, movie = load_dataset(ml100k);

In [3]:
rating = rating |> TableOperations.transform(Dict(:rating=>x->1.))

TableOperations.Transforms{true, CSV.File{false}, Dict{Symbol, var"#1#2"}}(CSV.File("/Users/keisuke.yanagi/workspace/Recommender.jl/src/dataset/../../dataset/movielens100k/u.data"):
Size: 100000 x 4
Tables.Schema:
 :userid     Int64
 :movieid    Int64
 :rating     Int64
 :timestamp  Int64, Dict(:rating => var"#1#2"()))

In [4]:
Random.seed!(1234);
train_valid_table, test_table = ratio_split(rating, 0.8)

train_table, valid_table = ratio_split(train_valid_table, 0.8)
length(Tables.rows(train_table)), length(Tables.rows(valid_table)), length(Tables.rows(test_table))


(64000, 16000, 20000)

In [5]:
prec10 = MeanPrecision(10)
recall10 = MeanRecall(10)
ndcg10 = MeanNDCG(10)
metrics = [prec10, recall10, ndcg10]

3-element Vector{Recommender.MeanMetric}:
 Recommender.MeanMetric{Recommender.Precision}(Recommender.Precision(10, "precision"))
 Recommender.MeanMetric{Recommender.Recall}(Recommender.Recall(10, "recall"))
 Recommender.MeanMetric{Recommender.NDCG}(Recommender.NDCG(10, "ndcg"))

In [6]:
using TreeParzen

In [7]:
space = Dict(
    :topk=>HP.QuantUniform(:topk, 10., 500., 1.),
    :shrink=>HP.LogUniform(:shrink, log(1e-3), log(1e3)),
    :weighting=>HP.Choice(:weighting, 
        [
            Dict(:weighting=>:dummy, :weighting_at_inference=>false),
            Dict(:weighting=>:tfidf, :weighting_at_inference=>false),
            Dict(:weighting=>:bm25, :weighting_at_inference=>HP.Choice(:weighting_at_inference, [true, false]))
        ]
    ),
    :normalize=>HP.Choice(:normalize, [true, false]),
    :normalize_similarity=>HP.Choice(:normalize_similarity, [true, false])
)

Dict{Symbol, TreeParzen.Types.AbstractDelayed} with 5 entries:
  :weighting            => Choice(Param(:weighting, RandIndex(3)), Dict{Symbol,…
  :topk                 => QuantUniform(:topk, QuantUniform(10.0, 500.0, 1.0))
  :normalize            => Choice(Param(:normalize, RandIndex(2)), Bool[1, 0])
  :normalize_similarity => Choice(Param(:normalize_similarity, RandIndex(2)), B…
  :shrink               => LogUniform(:shrink, LogUniform(-6.90776, 6.90776))

In [8]:
function invert_output(params)
    k = convert(Int, params[:topk])
    model = ItemkNN(k, params[:shrink],params[:weighting][:weighting],params[:weighting][:weighting_at_inference],params[:normalize],params[:normalize_similarity])
    result = evaluate_u2i(model, train_table, valid_table, metrics, 10, col_user=:userid, col_item=:movieid, col_rating=:rating, drop_history=true)
    @info params, result
    return -result[:ndcg]
end

invert_output (generic function with 1 method)

In [9]:
best = fmin(invert_output, space, 100, logging_interval=-1)

┌ Info: (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :bm25, :weighting_at_inference => true), :topk => 244.0, :normalize => false, :normalize_similarity => true, :shrink => 0.08889649400194267), (recall = 0.16726506459396742, precision = 0.19617021276595664, ndcg = 0.2352782433710405))
└ @ Main In[8]:5
┌ Info: (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :dummy, :weighting_at_inference => false), :topk => 439.0, :normalize => true, :normalize_similarity => false, :shrink => 255.10416104188548), (recall = 0.14910090825488115, precision = 0.17351063829787158, ndcg = 0.21904387894039185))
└ @ Main In[8]:5
┌ Info: (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :tfidf, :weighting_at_inference => false), :topk => 203.0, :normalize => false, :normalize_similarity => true, :shrink => 0.0012425645571008473), (recall = 0.16640877284141148, precision = 0.18180851063829703, ndcg = 0.2285782440878047))
└ @ Main In[8]:5
┌ Info: (Dict{Symbol, 

Dict{Symbol, Any} with 5 entries:
  :weighting            => Dict{Symbol, Any}(:weighting=>:bm25, :weighting_at_i…
  :topk                 => 358.0
  :normalize            => true
  :normalize_similarity => true
  :shrink               => 0.00258278

In [10]:
best_model = ItemkNN(convert(Int, best[:topk]), best[:shrink],best[:weighting][:weighting],best[:weighting][:weighting_at_inference],best[:normalize],best[:normalize_similarity])
evaluate_u2i(best_model, train_valid_table, test_table, metrics, 10, col_user=:userid, col_item=:movieid, col_rating=:rating, drop_history=true)

(recall = 0.21803020561045208,
 precision = 0.3284199363732762,
 ndcg = 0.39856188059055203,)